<a href="https://colab.research.google.com/github/ArianeMora/enzyme-tk/blob/main/BLAST_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BLAST example with enzymetk



In [3]:
! pip install enzymetk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.8 MB/s eta 0:00:00


In [24]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [25]:
! conda install -c bioconda -c conda-forge diamond -y
! conda update diamond

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - diamond


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2026.1.4   |       hbd8a1cb_0         143 KB  conda-forge
    certifi-2026.1.4           |     pyhd8ed1ab_0         147 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    diamond-2.1.19             |       h13889ed_0         2.9 MB  bioconda
    libsqlite-3.51.2           |       h0c1763c_0         917 KB  conda-forge
    openssl-3.6.0              |       h26f9b46_0         3.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         8.2 MB

The following NEW packages will be INSTALLED:

  diamond            

In [3]:
import logging
from enzymetk.step import Step
import pandas as pd
import numpy as np
from multiprocessing.dummy import Pool as ThreadPool
from tempfile import TemporaryDirectory
import os
import subprocess
import random
import string
from tqdm import tqdm


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


class BLAST(Step):

    def __init__(self, id_col: str, sequence_col: str, label_col=None, database=None,
                 mode='blastp', args=None, tmp_dir=None, num_threads=1,
                 env_name = 'enzymetk', venv_name = None):
        self.id_col = id_col
        self.seq_col = sequence_col
        self.label_col = label_col  # This is whether it is query or reference
        self.mode = mode
        self.database = database
        self.args = args
        self.tmp_dir = tmp_dir
        self.num_threads = num_threads
        self.env_name = env_name
        self.venv_name = venv_name
        self.diamond = 'diamond' # Use for conda - if installed via conda otherwise this will run the
        if self.database is None and self.label_col is None:
            raise ValueError('Database is not set, you can pass a database that you have already created see diamond for more information or the sequences \
                             as part of your dataframe and pass the label column (this needs to have two values: reference and query) reference \
                             refers to sequences that you want to search against and query refers to sequences that you want to search for.')

        super().__init__()

    def install(self, blast_link=None, os_version='linux', env_args=None):
        # install conda env and blast in conda
        if not blast_link:
            if os_version == 'linux':
                blast_link = 'http://github.com/bbuchfink/diamond/releases/download/v2.1.19/diamond-linux64.tar.gz'
            else:
                raise Exception(
                        f"OS version {os_version} not supported yet. "
                        "Please provide a link to a diamond binary tar.gz file "
                        "or install via conda and set the conda env."
                    )

        cmd = ['wget', blast_link]
        self.run(cmd)
        cmd = ['tar', 'xzf', 'diamond-linux64.tar.gz']
        self.run(cmd)
        # Set it to be in the local folder
        self.diamond = './diamond'
        self.venv = None
        self.conda = None

    def __execute(self, data: list) -> np.array:
        df, tmp_dir = data
        tmp_label = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
        query_fasta = os.path.join(tmp_dir, f'{tmp_label}_query.fasta')
        ref_fasta = os.path.join(tmp_dir, f'{tmp_label}_ref.fasta')
        db_label = os.path.join(tmp_dir, f'{tmp_label}_db')
        # write fasta file which is the input for proteinfer
        if self.label_col is not None:
            with open(query_fasta, 'w+') as fout:
                query_df = df[df[self.label_col] == 'query']
                print(query_df)
                for entry, seq in query_df[[self.id_col, self.seq_col]].values:
                    fout.write(f'>{entry.strip()}\n{seq.strip()}\n')

            with open(ref_fasta, 'w+') as fout:
                query_df = df[df[self.label_col] == 'reference']
                print(query_df)
                for entry, seq in query_df[[self.id_col, self.seq_col]].values:
                    fout.write(f'>{entry.strip()}\n{seq.strip()}\n')
            # Make the DB first
            db_label = os.path.join(tmp_dir, f'{tmp_label}_refdb')
            subprocess.run([self.diamond, 'makedb', '--in', ref_fasta, '-d', db_label], check=True)
        else:
            with open(query_fasta, 'w+') as fout:
                for entry, seq in df[[self.id_col, self.seq_col]].values:
                    fout.write(f'>{entry.strip()}\n{seq.strip()}\n')
            if os.path.exists(self.database):
                # Here we're assuming they're passing a database as a fasta file
                subprocess.run([self.diamond, 'makedb', '--in', self.database, '-d', db_label], check=True)
            else:
                db_label = self.database

        # Running Clustal Omega on the generated FASTA file
        matches_filename = os.path.join(tmp_dir, f'{tmp_label}_matches.tsv')
        cmd = ['diamond', self.mode]
        if self.args is not None:
            cmd.extend(self.args)
        cmd.extend(['-d', db_label, '-q', query_fasta, '-o', matches_filename])
        print(cmd)
        self.run(cmd)
        df = pd.read_csv(matches_filename, sep='\t', header=None)
        print(df)
        df.columns = ['query', 'target', 'sequence identity', 'length', 'mismatch', 'gapopen', 'query start', 'query end', 'target start', 'target end', 'e-value', 'bitscore']
        return df

    def execute(self, df: pd.DataFrame) -> pd.DataFrame:
        with TemporaryDirectory() as tmp_dir:
            tmp_dir = self.tmp_dir if self.tmp_dir is not None else tmp_dir
            if self.num_threads > 1:
                output_filenames = []
                df_list = np.array_split(df, self.num_threads)
                for df_chunk in tqdm(df_list):
                    try:
                        output_filenames.append(self.__execute([df_chunk, tmp_dir]))
                    except Exception as e:
                         logger.error(f"Error in executing ESM2 model: {e}")
                         continue
                df = pd.DataFrame()
                for sub_df in output_filenames:
                    df = pd.concat([df, sub_df])
                return df

            else:
                return self.__execute([df, tmp_dir])



In [4]:
import pandas as pd

# This should be where you downloaded the data from zotero, there is a folder in there called AS_inference
# This contains the models and the data needed to run the tool
id_col = 'Entry'
seq_col = 'Sequence'
label_col = 'label'
rows = [['AXE2_TALPU', 'query', 'MHSKFFAASLLGLGAAAIPLEGVMEKRSCPAIHVFGARETTASPGYGSSSTVVNGVLSAYPGSTAEAINYPACGGQSSCGGASYSSSVAQGIAAVASAVNSFNSQCPSTKIVLVGYSQGGEIMDVALCGGGDPNQGYTNTAVQLSSSAVNMVKAAIFMGDPMFRAGLSYEVGTCAAGGFDQRPAGFSCPSAAKIKSYCDASDPYCCNGSNAATHQGYGSEYGSQALAFVKSKLG'],
        ['AXE2_TALPU', 'reference', 'MHSKFFAASLLGLGAAAIPLEGVMEKRSCPAIHVFGARETTASPGYGSSSTVVNGVLSAYPGSTAEAINYPACGGQSSCGGASYSSSVAQGIAAVASAVNSFNSQCPSTKIVLVGYSQGGEIMDVALCGGGDPNQGYTNTAVQLSSSAVNMVKAAIFMGDPMFRAGLSYEVGTCAAGGFDQRPAGFSCPSAAKIKSYCDASDPYCCNGSNAATHQGYGSEYGSQALAFVKSKLG'],
        ['AXE2_GEOSE', 'reference', 'MKIGSGEKLLFIGDSITDCGRARPEGEGSFGALGTGYVAYVVGLLQAVYPELGIRVVNKGISGNTVRDLKARWEEDVIAQKPDWVSIMIGINDVWRQYDLPFMKEKHVYLDEYEATLRSLVLETKPLVKGIILMTPFYIEGNEQDPMRRTMDQYGRVVKQIAEETNSLFVDTQAAFNEVLKTLYPAALAWDRVHPSVAGHMILARAFLREIGFEWVRSR'],
        ['AXE7A_XYLR2', 'referece', 'MFNFAPKQTTEMKKLLFTLVFVLGSMATALAENYPYRADYLWLTVPNHADWLYKTGERAKVEVSFCLYGMPQNVEVAYEIGPDMMPATSSGKVTLKNGRAVIDMGTMKKPGFLDMRLSVDGKYQHHVKVGFSPELLKPYTKNPQDFDAFWKANLDEARKTPVSVSCNKVDKYTTDAFDCYLLKIKTDRRHSIYGYLTKPKKAGKYPVVLCPPGAGIKTIKEPMRSTFYAKNGFIRLEMEIHGLNPEMTDEQFKEITTAFDYENGYLTNGLDDRDNYYMKHVYVACVRAIDYLTSLPDWDGKNVFVQGGSQGGALSLVTAGLDPRVTACVANHPALSDMAGYLDNRAGGYPHFNRLKNMFTPEKVNTMAYYDVVNFARRITCPVYITWGYNDNVCPPTTSYIVWNLITAPKESLITPINEHWTTSETNYTQMLWLKKQVK'],
        ['A0A0B8RHP0_LISMN', 'reference', 'MKKLLFLGDSVTDAGRDFENDRELGHGYVKIIADQLEQEDVTVINRGVSANRVADLHRRIEADAISLQPDVVTIMIGINDTWFSFSRWEDTSVTAFKEVYRVILNRIKTETNAELILMEPFVLPYPEDRKEWRGDLDPKIGAVRELAAEFGATLIPLDGLMNALAIKHGPTFLAEDGVHPTKAGHEAIASTWLEFTK']]
df = pd.DataFrame(rows, columns=[id_col, label_col, seq_col])
blast = BLAST(id_col, seq_col, label_col)
# blast.install() since we installed with conda we don't need to run this - this is if you don't want to conda install

In [5]:
! mkdir tmp

mkdir: cannot create directory ‘tmp’: File exists


In [6]:
blast.tmp_dir = 'tmp'
blast.execute(df)

diamond v2.1.19.173 (C) Max Planck Society for the Advancement of Science, Benjamin J. Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 2
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: tmp
#Target sequences to report alignments for: 25
Opening the database...  [0s]
Database: tmp/z9yaQgjVyD_refdb (type: Diamond database, sequences: 3, letters: 650)
Block size = 2000000000
Opening the input file...  [0s]
Opening the output file...  [0s]
Loading query sequences...  [0s]
Masking queries...  [0.003s]
Algorithm: Double-indexed
Building query histograms...  [0s]
Seeking in database...  [0s]
Loading reference sequences...  [0s]
Masking reference...  [0.004s]
Initializing temporary storage...  [0.001s]
Building reference histograms...  [0s]
Allocating buffers...  [0s]
Processing query block 1,

        Entry  label                                           Sequence
0  AXE2_TALPU  query  MHSKFFAASLLGLGAAAIPLEGVMEKRSCPAIHVFGARETTASPGY...
              Entry      label  \
1        AXE2_TALPU  reference   
2        AXE2_GEOSE  reference   
4  A0A0B8RHP0_LISMN  reference   

                                            Sequence  
1  MHSKFFAASLLGLGAAAIPLEGVMEKRSCPAIHVFGARETTASPGY...  
2  MKIGSGEKLLFIGDSITDCGRARPEGEGSFGALGTGYVAYVVGLLQ...  
4  MKKLLFLGDSVTDAGRDFENDRELGHGYVKIIADQLEQEDVTVINR...  
['diamond', 'blastp', '-d', 'tmp/z9yaQgjVyD_refdb', '-q', 'tmp/z9yaQgjVyD_query.fasta', '-o', 'tmp/z9yaQgjVyD_matches.tsv']
--------------------------------------------------------------------------------
Running command	diamond blastp -d tmp/z9yaQgjVyD_refdb -q tmp/z9yaQgjVyD_query.fasta -o tmp/z9yaQgjVyD_matches.tsv	
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                

,query,target,sequence identity,length,mismatch,gapopen,query start,query end,target start,target end,e-value,bitscore
0,AXE2_TALPU,AXE2_TALPU,100,234,0,0,1,234,1,234,7.800000e-168,451
